In [32]:
from search import *
from game import *
from util import *
from pacman import *
import util, layout
import sys, types, time, random, os



In [39]:
# tiny genetic programming by © moshe sipper, www.moshesipper.com
from random import random, randint, seed
from statistics import mean
from copy import deepcopy

POP_SIZE        = 60   # population size
MIN_DEPTH       = 2    # minimal initial random tree depth
MAX_DEPTH       = 5    # maximal initial random tree depth
GENERATIONS     = 250  # maximal number of generations to run evolution
TOURNAMENT_SIZE = 5    # size of tournament for tournament selection
XO_RATE         = 0.8  # crossover rate 
PROB_MUTATION   = 0.2  # per-node mutation probability 



def add(x, y): return x + y
def sub(x, y): return x - y
def mul(x, y): return x * y
FUNCTIONS = [add, sub, mul]
TERMINALS = ['x', -2, -1, 0, 1, 2] 

class GPTree:
    def __init__(self, data = None, left = None, right = None):
        self.data  = data
        self.left  = left
        self.right = right
        
    def node_label(self): # string label
        if (self.data in FUNCTIONS):
            return self.data.__name__
        else: 
            return str(self.data)
    
    def print_tree(self, prefix = ""): # textual printout
        print("%s%s" % (prefix, self.node_label()))        
        if self.left:  self.left.print_tree (prefix + "   ")
        if self.right: self.right.print_tree(prefix + "   ")

    def compute_tree(self, x): 
        if (self.data in FUNCTIONS): 
            return self.data(self.left.compute_tree(x), self.right.compute_tree(x))
        elif self.data == 'x': return x
        else: return self.data
    
    def random_tree(self, grow, max_depth, depth = 0): # create random tree using either grow or full method
        if depth < MIN_DEPTH or (depth < max_depth and not grow): 
            self.data = FUNCTIONS[randint(0, len(FUNCTIONS)-1)]
        elif depth >= max_depth:   
            self.data = TERMINALS[randint(0, len(TERMINALS)-1)]
        else: # intermediate depth, grow
            if random () > 0.5: 
                self.data = TERMINALS[randint(0, len(TERMINALS)-1)]
            else:
                self.data = FUNCTIONS[randint(0, len(FUNCTIONS)-1)]
        if self.data in FUNCTIONS:
            self.left = GPTree()          
            self.left.random_tree(grow, max_depth, depth = depth + 1)            
            self.right = GPTree()
            self.right.random_tree(grow, max_depth, depth = depth + 1)
        

In [46]:
tree = GPTree()
tree.random_tree(True, 5)
tree.print_tree()

mul
   mul
      -1
      sub
         mul
            2
            -2
         1
   add
      sub
         mul
            mul
               -2
               1
            1
         -2
      x


In [47]:
n = Directions.NORTH
s = Directions.SOUTH
e = Directions.EAST
w = Directions.WEST
actions = [w,w,w,w,s,w]

class MyGoWestAgent(Agent):
    "An agent that goes West until it can't."
    

    def getAction(self, state):
        "The agent receives a GameState (defined in pacman.py)."
        print(state.getLegalPacmanActions())
        print(state.getPacmanState())

        print(state.getPacmanPosition())
        print(state.getNumFood())
        print(state.getScore())
        action = s
        if len(actions) > 0:
            action = actions.pop(0)
        if action in state.getLegalPacmanActions():
            print(state.generatePacmanSuccessor(action).getGhostPositions())
            return action
        else:
            return Directions.STOP

In [48]:
args = readCommand(['--layout', 'smallClassic', '--pacman', 'GoWestAgent', '-t']) # Get game components based on input
gameLayout = layout.getLayout( 'smallClassic' )

pacman = MyGoWestAgent()

ghostType = loadAgent('RandomGhost', True)
ghosts = [ghostType( i+1 ) for i in range( 4 )]

import textDisplay
textDisplay.SLEEP_TIME = 0.1
gameDisplay = textDisplay.PacmanGraphics()
# runGames( **args )
runGames( gameLayout, pacman, ghosts, gameDisplay, 1, False, numTraining = 0, catchExceptions=False, timeout=30 )

%%%%%%%%%%%%%%%%%%%%
%......%G  G%......%
%.%%...%%  %%...%%.%
%.%o.%........%.o%.%
%.%%.%.%%%%%%.%.%%.%
%........<.........%
%%%%%%%%%%%%%%%%%%%%
Score: 0

['West', 'Stop', 'East']
Pacman: (x,y)=(9, 1), Stop
(9, 1)
55
0.0
[(8, 5), (11, 5)]
%%%%%%%%%%%%%%%%%%%%
%......% GG %......%
%.%%...%%  %%...%%.%
%.%o.%........%.o%.%
%.%%.%.%%%%%%.%.%%.%
%.......> .........%
%%%%%%%%%%%%%%%%%%%%
Score: 9

['West', 'Stop', 'East']
Pacman: (x,y)=(8, 1), West
(8, 1)
54
9.0
[(9.0, 5.0), (10.0, 5.0)]
%%%%%%%%%%%%%%%%%%%%
%......% GG %......%
%.%%...%%  %%...%%.%
%.%o.%........%.o%.%
%.%%.%.%%%%%%.%.%%.%
%......>  .........%
%%%%%%%%%%%%%%%%%%%%
Score: 18

['West', 'Stop', 'East']
Pacman: (x,y)=(7, 1), West
(7, 1)
53
18.0
[(10.0, 5.0), (9.0, 5.0)]
%%%%%%%%%%%%%%%%%%%%
%......%G   %......%
%.%%...%% G%%...%%.%
%.%o.%........%.o%.%
%.%%.%.%%%%%%.%.%%.%
%.....>   .........%
%%%%%%%%%%%%%%%%%%%%
Score: 27

['West', 'Stop', 'East', 'North']
Pacman: (x,y)=(6, 1), West
(6, 1)
52
27.0
[(10.0, 4.0), (8.0, 5.0)]

[<game.Game instance at 0x7f8799f25aa0>]